In [1]:
!ln -s /home/data /habitat-api/data

In [2]:
!ln -s /habitat-api /home/habitat-api

In [3]:
!cp ./configs/default.py /habitat-api/habitat/config/default.py

In [4]:
!pip install tokenizer stable-baselines

     |################################| 112kB 896kB/s eta 0:00:01
     |################################| 235kB 2.9MB/s eta 0:00:01
     |################################| 296kB 20.6MB/s eta 0:00:01
     |################################| 2.4MB 48.7MB/s eta 0:00:01
     |################################| 2.8MB 66.4MB/s eta 0:00:01


In [5]:
import os
import gym
import numpy as np

from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import PPO2

from environment import environments
import habitat

/opt/conda/envs/habitat/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/habitat/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/habitat/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/habitat/lib/python3.6/site-packages/tensorflow/python/framework

In [6]:
import sys
sys.path.append('./agents/')
from custom_local_policy import LocalPolicy

In [7]:
from stable_baselines.common.policies import register_policy

In [9]:
if __name__ == '__main__':
    num_cpu = 4
    n_steps = 30
    n_updates = 2
    log_dir = './tensorboard'
    config = habitat.get_config("./configs/pointnav_kozub.yaml")
    env = SubprocVecEnv([lambda: environments.NavRLEnvLocalPolicy(config) for i in range(num_cpu)])
    print('environments have been created.')
    model = PPO2(policy=LocalPolicy, 
                 env=env,
                 gamma=0.99,
                 cliprange = 0.3, #if wasn't mensioned in ANM paper.
                 verbose=1,
                 n_steps=n_steps,
                 learning_rate=25e-6,
                 ent_coef=1e-3,
                 vf_coef=0.5,
                 tensorboard_log=log_dir,
                 policy_kwargs = {'rgb_shape': [config.SIMULATOR.RGB_SENSOR.HEIGHT, 
                                                config.SIMULATOR.RGB_SENSOR.WIDTH, 3]}
                 )
    model.learn(total_timesteps=num_cpu*n_steps*n_updates)
    print('learning complete!')

environments have been created.
------------------------------------
| approxkl           | 0.07535396  |
| clipfrac           | 0.47500002  |
| explained_variance | -1.89       |
| fps                | 2           |
| n_updates          | 1           |
| policy_entropy     | 0.78530174  |
| policy_loss        | -0.00500492 |
| serial_timesteps   | 30          |
| time_elapsed       | 4.29e-06    |
| total_timesteps    | 120         |
| value_loss         | 1418.1132   |
------------------------------------
------------------------------------
| approxkl           | 0.084124364 |
| clipfrac           | 0.5395833   |
| explained_variance | -0.782      |
| fps                | 2           |
| n_updates          | 2           |
| policy_entropy     | 0.82077676  |
| policy_loss        | 0.022173405 |
| serial_timesteps   | 60          |
| time_elapsed       | 58.4        |
| total_timesteps    | 240         |
| value_loss         | 1388.598    |
------------------------------------
learni

In [49]:
model_path = './LP_PPO_model'
model.save(save_path =model_path)
del model
model = PPO2.load(load_path=model_path)

Loading a model without an environment, this model cannot be trained until it has a valid environment.


In [50]:
obs = env.reset()

counter = 0
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    counter += 1
    print(counter)
    if dones.any():
        break
print(counter)

env.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
!pip install grpcio==1.24.3
!pip uninstall --yes tb-nightly
%load_ext tensorboard
%tensorboard --logdir ./tensorboard/

In [17]:
for i, layer_size in enumerate([128, 128, 128]):
    print(i, layer_size)

0 128
1 128
2 128
